<a href="https://colab.research.google.com/github/GY20150609/Python/blob/master/%E9%97%AD%E5%8C%85%26%E8%A3%85%E9%A5%B0%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.闭包

非人话：Objects(对象) are data with methods attached. Closures(闭包) are functions with data attached.

翻 译：所谓对象(Object)，指的是附带相应方法的数据。那么相对而言，闭包(closure)指的则是附带相应数据的函数。

人 话：闭包函数能够引用一些并不在当前函数定义代码块中定义的变量。

**1.1 python2.X 中的的闭包**

只能获取闭包中自由变量的值，但无法改变其值。

In [0]:
#EXAMPLE
def constant_adder(x):
    #自由变量constant位于函数代码块外面
    constant = x
    def adder(y):
        y = y + constant
        return y
    return adder
  
f1 = constant_adder(1)
f1(2)

3

**1.2 python3.X 中的的闭包**

Python3.0引入了**nonlocal**关键字，不仅能获取闭包中自由变量的值，还能改变其值

In [0]:
#EXAMPLE
def constant_adder(x):
    #自由变量constant位于函数代码块外面
    constant = x
    def adder(y):
        #改变自由变量的值
        nonlocal constant
        constant = 10
        y = y + constant
        return y
    return adder
  
f1 = constant_adder(1)
f1(2)

12

#2.装饰器(Decorator)-基础

Python中的函数(function)也是对象，可以作为参数传递，可以被赋值给另一个变量。

**2.1 函数对象**

*函数名只是对函数方法的引用，可以通过赋值达到调用该函数方法的目的，删除函数名不影响函数方法的调用。


In [0]:
#EXAMPLE
def add(x,y):
  return x + y

#函数赋值
add(1,2)
f = add
f(1,2)

#删除函数名
del add
f(1,2)   --正常执行
add(1,2) --报错

*函数可以非常灵活地在各种位置被定义，包括另一个函数内部。

In [0]:
#EXAMPLE
def adder():
  def add(x=1,y=2):
    return x + y
  return add()

adder()

3

**2.2 装饰器基本逻辑**

python中装饰器其实只是对其所装饰的函数的一层额外包装。即接受一个函数作为输入，然后定义另外一个包装函数在其执行前后加入另外一些逻辑，最终返回这个

包装函数。在装饰器中，我们可以完全不修改原有函数的情况下，执行所装饰的函数之外另外包装一些别的代码逻辑。

**Step1**: 定义需要装饰的函数

一般用于实现某种具体的功能

**Step2**: 构建最外层装饰函数

**Step3**: 在装饰函数中定义一个包装函数

这个函数将负责在我们所要装饰的函数（调用）前后文中添加我们需要的代码逻辑

**Step4**：包装函数作为装饰器函数的返回

In [0]:
# 装饰器就是接受一个函数作为输入，并返回另一个函数的函数
def basic_decorator_logic(func_to_decorate): 
    # 定义包装函数
    def the_wrapper_around_the_original_function():
        # 前逻辑
        print('Before the original function runs')

        # 调用原始函数，体现了python中闭包的概念
        func_to_decorate()

        # 后逻辑
        print('After the original function runs')

    # 返回包装函数
    # the_wrapper_around_the_original_function`包含需要在被装饰函数执行前后需要添加的逻辑以及被包装函数的执行
    # 这里返回的是动态定义的包装函数对象本身，而不是包装函数的执行结果
    return the_wrapper_around_the_original_function

**2.3 装饰器的使用**

python中使用装饰器往往是为了在后文中完全用装饰过后的函数替代我们原本定义的函数，我们可以将装饰过后的函数赋值给原函数对应的变量名，从而在代码下

文中实现永久替换

In [0]:
## 显式使用

def function_we_want_to_decorate():
    print('This is a function that is going to be decorated, we can add additional execution logic without changing the function')

function_we_want_to_decorate()

# 我们只需要将`function_we_want_to_decorate`作为参数传入我们上面定义的装饰器函数中，就可以获得一个被包装过的新函数。
# 这一新函数中包含了一些我们额外添加的逻辑
function_we_want_to_decoratre = basic_decorator_logic(function_we_want_to_decorate)

function_we_want_to_decoratre()

This is a function that is going to be decorated, we can add additional execution logic without changing the function
Before the original function runs
This is a function that is going to be decorated, we can add additional execution logic without changing the function
After the original function runs


In [0]:
## 隐式使用 装饰器语法@  声明后待包装函数不能直接调用

@basic_decorator_logic
def function_we_want_to_decorate():
    print('This is a function that is going to be decorated, we can add additional execution logic without changing the function')

**2.4 多个装饰器的使用**

多层装饰器将被从里到外执行，也就是对于同一个函数定义上方的装饰器，最上面一行的装饰器将被最后套用，而最下面一行的装饰器将被最先套用

In [0]:
def outer_decorator(func):
    def wrapper():
        print('outer_wrapper_before')
        func()
        print('outer_wrapper_aftrer')
    return wrapper

def inner_decorator(func):
    def wrapper():
        print('inner_wrapper_before')
        func()
        print('inner_wrapper_after')
    return wrapper

def hotpot(sentence='I love hotpot!'):
    print(sentence)

def func():
  print('I love hotpot!')
# output: I love hotpot!

func = outer_decorator(inner_decorator(func))
func()

outer_wrapper_before
inner_wrapper_before
I love hotpot!
inner_wrapper_after
outer_wrapper_aftrer


In [0]:
@outer_decorator
@inner_decorator
def beijing_duck(sentence='I love beijing duck!'):
    print(sentence)

beijing_duck()

outer_wrapper_before
inner_wrapper_before
I love beijing duck!
inner_wrapper_after
outer_wrapper_aftrer


In [0]:
@inner_decorator
@outer_decorator
def gopnik(sentence='Gopnik!'):
    print('adidas, adidas hard bass and ignoring gravity are the 3 most important factors for gopnik dance! Check it out on bilibili and you will laugh your ass off.')

gopnik()

inner_wrapper_before
outer_wrapper_before
adidas, adidas hard bass and ignoring gravity are the 3 most important factors for gopnik dance! Check it out on bilibili and you will laugh your ass off.
outer_wrapper_aftrer
inner_wrapper_after


#3.装饰器(Decorator)-进阶

**3.1 对带传参的函数进行装饰**

由于装饰器事实上返回了一个新的函数来代替我们需要装饰的函数，所以我们只需要确保在装饰器中返回的新的函数与原函数所接受的参数格式一致即可

In [0]:
def decorator_for_func_with_arguments(func_to_decorate):
    def wrapper_that_passes_through_arguments(arg1, arg2):
        print('I got args! Look: {} {}'.format(arg1, arg2))
        func_to_decorate(arg1, arg2)
    return wrapper_that_passes_through_arguments

#等价于 print_full_name = decorator_for_func_with_arguments(print_full_name)
@ decorator_for_func_with_arguments   
def print_full_name(first_name, last_name):
    print('I am {} {}'.format(first_name, last_name))
    
print_full_name('guo','yu')

I got args! Look: guo yu
I am guo yu


**3.2 对类方法进行装饰**

类方法事实上与函数一样，只是固定接收当前实例的引用作为第一个参数，一般标记为self。那么能够装饰类方法的装饰器事实上也可以用与问题一中一致的方法实

现，只不过我们总是要确保返回的函数所接收的第一个参数也是当前实例的引用（self）即可。

In [0]:
def decorator_for_instance_method(method_to_decorate):
    def wrapper(self, bonus):
        # 升职加薪，奖金增加一倍
        bonus = bonus * 2
        return method_to_decorate(self, bonus)
    return wrapper

class Salary(object):
    def __init__(self):
        self.base = 666

    @decorator_for_instance_method
    def total_compensation(self, bonus):
        print('Congrats! You got a total compensation of {}'.format(self.base * 12 + bonus))
        
ss = Salary()
ss.total_compensation(10)


Congrats! You got a total compensation of 8012


**3.3 对接收任意数目参数的函数进行装饰**

"* args"  && "** kwargs"

In [0]:
#无参函数

def decorator_passing_arbitrary_arguments(function_to_decorate):
    def wrapper_with_arbitrary_arguments(*args, **kwargs):
        print('Received arguments as following')
        print(args)
        print(kwargs)

        function_to_decorate(*args, **kwargs)

    return wrapper_with_arbitrary_arguments

@decorator_passing_arbitrary_arguments
def function_with_no_argument():
    print('This function does not have any argument')

function_with_no_argument()


Received arguments as following
()
{}
This function does not have any argument


In [0]:
#有参函数

@decorator_passing_arbitrary_arguments
def function_with_arguments(a, b, c):
    print('This function has arguments')

function_with_arguments(1,2,3)


Received arguments as following
(1, 2, 3)
{}
This function has arguments


In [0]:
#带字典参数的函数

@decorator_passing_arbitrary_arguments
def function_with_named_arguments(a, b, c, name):
    print('{}, {}, {}'.format(a, b, c))
    print('{}'.format(name))

function_with_named_arguments(1, 2, 3, name='robot')


Received arguments as following
(1, 2, 3)
{'name': 'robot'}
1, 2, 3
robot


In [0]:
#类方法，实例对象也会作为一个参数存在args中

class Salary(object):
    def __init__(self):
        self.base = 666

    @decorator_passing_arbitrary_arguments
    def total_compensation(self, bonus):
        print('Congrats! You got a total compensation of {}'.format(self.base * 12 + bonus))

salary = Salary()
salary.total_compensation(2048)

Received arguments as following
(<__main__.Salary object at 0x7fc8cb364dd8>, 2048)
{}
Congrats! You got a total compensation of 10040


**3.4 装饰函数传入参数**

装饰器工厂函数——动态生成装饰器

实现一个能够返回装饰器的函数来替装饰器接收参数，并使用闭包的方法来将这些参数传递到装饰器中。

In [0]:
#显式用法

def decorator_maker():
    print('This is a factory generating decorators on the fly. This function is called once we want a fresh decorator.')

    def my_decorator(func):
        print('This is the decorator generated on the fly. This function is called when the decoration happens.')
        # 类似地，我们还是在装饰器中定义一个wrapper还包装原函数
        def wrapper():
            print('This is the wrapper around the decorated function. This function is called once the decorated function is called.')
            return func()

        return wrapper

    print('The decorator function created on the fly is returned.')
    return my_decorator

def func():
    print('This is the function decorated.')

fresh_decorator = decorator_maker()

func = fresh_decorator(func)

func()

In [0]:
# 隐式使用

@decorator_maker()
def func():
    print('This is the function decorated.')

func()